<a href="https://colab.research.google.com/github/natelson/python/blob/main/web_scraping/download_info_about_cars_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=green>1. WEB SCRAPING FOR BEGINNERS

# 1.1. What is web scraping?

> *Web scraping* is the term used to extract data from web sites, there are numerous reasons to do this.

> For example, suppose you want to build a machine learning model that recognizes in a photo whether or not it has a car, to train your model you will need hundreds of photos of cars to train the model.

> Then you can create a bot that accesses car sales websites and downloads these photos to train your model later.

> The idea here is to show you how to do this in python using some libraries for this, BeatifulSoup to work with the html we are analyzing. Urlib to request and download the content of pages and images and Pandas to transform this information into structured data so that we can store and work with them.

# 1.2. Importing the libraries

> For this example of web scraping, I used one fake website about cars, and the goal is download images of cars and extract some information about them.

> The first step is importing the libraries, like the code below.


In [539]:
import bs4
import urllib.request as urllib_request
import pandas

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pandas.__version__)

BeautifulSoup -> 4.6.3
urllib -> 3.7
pandas -> 1.3.5


---
# <font color=green>2. Working with requests

## 2.1. Getting the HTML content of a website

> For the download of code html of a page web, I used the library urlib.request, the link to documentation and the sample of use is below.

# urllib.request
## https://docs.python.org/3/library/urllib.html

In [540]:
from urllib.request import urlopen

url = 'https://alura-site-scraping.herokuapp.com/hello-world.php'

response = urlopen(url)
html = response.read()
print(html[:500])

b'<!DOCTYPE html>\r\n<html lang="pt-br">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\r\n\r\n    <title>Alura Motors</title>\r\n\t<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">\r\n\t<link rel="stylesheet" href="css/styles.css" media="all">\r\n\r\n\t<script src="https://code'


## 2.2. Some sites not allow bots, what you do?

> Some sites, not allow bots, in this case for “lie” to website is necessary to send information about the user-agent in header of request. The information about user-agent sends the server of website of type of browser is visiting.
> When our working with bots the web scraping is important create a behavior similar a user browsing in a web browser like the chrome. The user-agent have an important role this case.

In [541]:
from urllib.request import Request, urlopen
url = 'https://alura-site-scraping.herokuapp.com'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

req = Request(url, headers = headers)

## 2.3. If it's possible to fail, it will fail
> When us to working with request a website, a large number of possible of error is possible. The website is down, is slow, the domain change, the access was blocked and the other situations.

> In this case it is important to deal with possible errors. The code below was created to deal with 3 different errors:

### 2.3.1 HTTP Status Code <> 2xxx

> Some times one website is not allowing the direct access and return some status code that raise errors in your application.
> For example, the website of a company of course is not allowed access without header, return the HTTP Code 403, the code below handle HTTPError type errors.
Note: For more information about http code visit https://developer.mozilla.org/en-US/docs/Web/HTTP/Status


In [542]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
url = 'https://alura.com.br/'
try:
    req = Request(url)
    response = urlopen(req)
    print(response.read())

except HTTPError as e:
    print(e.status, e.reason)


403 Forbidden


### 2.3.2 URL Errors

Some times one website is down or the domain is changed, in this case your request returns one URLError.

For this example, I change the url https://twitter.com for https://twitter.comx , such this domain not exist, the urlopen will return one urlerror

In [543]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
url = 'https://twitter.comx/'
try:
    req = Request(url)
    response = urlopen(req)
    html = response.read()

except HTTPError as e:
    print(e.status, e.reason)

except URLError as e:
  print(e.reason)

[Errno -2] Name or service not known


#### For more information about the library request visit https://docs.python.org/3/library/urllib.request.html#urllib.request.Request

## 2.4 String handling

> When working with web scraping is important transform the code html in some text more friendly for work with him.
> Below have some examples to convert the html code a text more friendly for work. 

> **The BeatifulSoup library will be working without these treatments, but sometimes some errors is solved with this approach.**

### Converting type bytes to string

In [544]:
from urllib.request import urlopen

url = 'https://alura-site-scraping.herokuapp.com/index.php'

response = urlopen(url)
html = response.read()
html[:500]

b'<!DOCTYPE html>\r\n<html lang="pt-br">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\r\n\r\n    <title>Alura Motors</title>\r\n\r\n\t<style>\r\n\t\t/*Regra para a animacao*/\r\n\t\t@keyframes spin {\r\n\t\t\t0% { transform: rotate(0deg); }\r\n\t\t\t100% { transform: rotate(360deg); }\r\n\t\t}\r\n\t\t/*Mudando o tamanho do icone de resposta*/\r\n\t\tdiv.glyphicon {\r\n\t\t\tcolor:#6B8E23;\r\n\t\t\tfont-size: 38px;\r\n\t\t}\r\n\t\t/*Classe que mostra a animacao \'spin\'*/\r\n\t\t.'

In [545]:
type(html)

bytes

In [546]:
html_text = html.decode('utf-8')

In [547]:
type(html_text)

str

### Remove tab characters, line breaks, etc.

In [548]:
html_text.split()[:50]

['<!DOCTYPE',
 'html>',
 '<html',
 'lang="pt-br">',
 '<head>',
 '<meta',
 'charset="utf-8">',
 '<meta',
 'name="viewport"',
 'content="width=device-width,',
 'initial-scale=1,',
 'shrink-to-fit=no">',
 '<title>Alura',
 'Motors</title>',
 '<style>',
 '/*Regra',
 'para',
 'a',
 'animacao*/',
 '@keyframes',
 'spin',
 '{',
 '0%',
 '{',
 'transform:',
 'rotate(0deg);',
 '}',
 '100%',
 '{',
 'transform:',
 'rotate(360deg);',
 '}',
 '}',
 '/*Mudando',
 'o',
 'tamanho',
 'do',
 'icone',
 'de',
 'resposta*/',
 'div.glyphicon',
 '{',
 'color:#6B8E23;',
 'font-size:',
 '38px;',
 '}',
 '/*Classe',
 'que',
 'mostra',
 'a']

In [549]:
" ".join(html_text.split())[:500]

'<!DOCTYPE html> <html lang="pt-br"> <head> <meta charset="utf-8"> <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"> <title>Alura Motors</title> <style> /*Regra para a animacao*/ @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } } /*Mudando o tamanho do icone de resposta*/ div.glyphicon { color:#6B8E23; font-size: 38px; } /*Classe que mostra a animacao \'spin\'*/ .loader { border: 16px solid #f3f3f3; border-radius: 50%; border'

### Eliminating white spaces between TAGS

In [550]:
" ".join(html_text.split()).replace('> <', '><')[:500]

'<!DOCTYPE html><html lang="pt-br"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><title>Alura Motors</title><style> /*Regra para a animacao*/ @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } } /*Mudando o tamanho do icone de resposta*/ div.glyphicon { color:#6B8E23; font-size: 38px; } /*Classe que mostra a animacao \'spin\'*/ .loader { border: 16px solid #f3f3f3; border-radius: 50%; border-top: '

### String handling function

In [551]:
def change_html_byte_to_text(html_byte):
    html_text = html_byte.decode('utf-8')
    html_text =  " ".join(html_text.split()).replace('> <', '><')
    return html_text

In [552]:
html = change_html_byte_to_text(html)

In [553]:
html[:500]

'<!DOCTYPE html><html lang="pt-br"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><title>Alura Motors</title><style> /*Regra para a animacao*/ @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } } /*Mudando o tamanho do icone de resposta*/ div.glyphicon { color:#6B8E23; font-size: 38px; } /*Classe que mostra a animacao \'spin\'*/ .loader { border: 16px solid #f3f3f3; border-radius: 50%; border-top: '

# <font color=green>3. Extracting information with BeautifulSoup

# 3.1. Understand the format of an HTML code

**HTML** (*HyperText Markup Language*) is a markup language made up of **tags** that determine the role that each part of the document will assume. The **tags** are formed by your name and attributes. Attributes are used to configure and also modify the default characteristics of a **tag**.

## Basic Structure

```html
<html>
    <head>
        <meta charset="utf-8" />
        <title>Alura Motors</title>
    </head>
    <body>
        <div id="container">
            <h1>Alura</h1>
            <h2 class="formato">Cursos de Tecnologia</h2>
            <p>Você vai estudar, praticar, discutir e aprender.</p>
            <a href="https://www.alura.com.br/">Clique aqui</a>
        </div>
    </body>
</html>
```

```<html>``` - determines the beginning of the document.

```<head>``` - header. Contains document information and settings.

```<body>``` - is the body of the document, where all the content is placed. This is the part visible in a browser.

## Most common tags

```<div>``` - Defines a division of the page. Can be formatted in different ways.

```<h1>, <h2>, <h3>, <h4>, <h5>, <h6>``` - Title markerss.

```<p>``` - Paragraph marker.

```<a>``` - hiperlink.

```<img>``` - image display.

```<table>``` - tables.

```<ul>, <li>``` - lists.


# 3.2. Creating a BeautifulSoup object

## https://www.crummy.com/software/BeautifulSoup/

### About parser: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#parser-installation

In [554]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')


In [555]:
print(type(soup))

print(soup.prettify()[:50])

<class 'bs4.BeautifulSoup'>
<!DOCTYPE html>
<html lang="pt-br">
 <head>
  <met


# 3.3. Accessing tags

In [556]:
# Read the tile - Solution 1
soup.html.head.title


<title>Alura Motors</title>

In [557]:
# Read the tile - Solution 2
soup.title

<title>Alura Motors</title>

In solution 1 it is possible to follow the structure of the html that was read by BeatifulSoup. In solution 2, we can directly put the name of the tag we want to read, in this case it will show the value of the first one it finds in the html structure that was read.

In [558]:
#Read the div - Solutions 1

soup.div.div.div.div

<div class="modal-header"><h5 class="modal-title" id="loadingModal_label"><span class="glyphicon glyphicon-refresh"></span>Aguarde... </h5></div>

In [559]:
#Read the div - Solutions 2
soup.div

<div aria-labelledby="loadingModal_label" class="modal fade" data-backdrop="static" id="loadingModal" role="dialog" tabindex="-1"><div class="modal-dialog" role="document"><div class="modal-content"><div class="modal-header"><h5 class="modal-title" id="loadingModal_label"><span class="glyphicon glyphicon-refresh"></span>Aguarde... </h5></div><div class="modal-body"><div class="alert" role="alert"><center><div class="loader" id="loader"></div><br/><h4><b id="loadingModal_content"></b></h4></center></div></div></div></div></div>

In this case, the result was different, in solution 1 we want to read the content of the fourth div, but in solution 2, when placing only Div, it brought the entire content of the first div.

# 3.4. Accessing tag content

In [560]:
#extract text ou contenf of tag - Solution 1
soup.html.head.title.get_text()


'Alura Motors'

In [561]:
#extract text ou contenf of tag - Solution 2
soup.html.head.title.getText()

'Alura Motors'

In [562]:
soup.div.div.div.div.get_text()

'Aguarde... '

In [563]:
#get all text
soup.get_text()

'Alura Motors /*Regra para a animacao*/ @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } } /*Mudando o tamanho do icone de resposta*/ div.glyphicon { color:#6B8E23; font-size: 38px; } /*Classe que mostra a animacao \'spin\'*/ .loader { border: 16px solid #f3f3f3; border-radius: 50%; border-top: 16px solid #3498db; width: 80px; height: 80px; -webkit-animation: spin 2s linear infinite; animation: spin 2s linear infinite; }  requisitar("https://alura-scraping.herokuapp.com/produtos?_page="+1+"&_limit=10") You need to enable JavaScript to run this app.Aguarde... Motors Hello WorldAnúnciosVeículos de Luxo Novos e Usados - Todas as Marcas246 veículos encontradosPágina 1 de 25LAMBORGHINI AVENTADORUSADOMotor 1.8 16vAno 1993 - 55.286 km► 4 X 4► Câmera de estacionamento► Controle de tração► Sensor de estacionamento...Belo Horizonte - MGR$ 338.000BMW M2USADOMotor 3.0 32vAno 2018 - 83.447 km► Câmera de estacionamento► Controle de estabilidade► Travas elétricas►

In [564]:
#get all text with a separator
soup.get_text(separator = '||')

'Alura Motors|| /*Regra para a animacao*/ @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } } /*Mudando o tamanho do icone de resposta*/ div.glyphicon { color:#6B8E23; font-size: 38px; } /*Classe que mostra a animacao \'spin\'*/ .loader { border: 16px solid #f3f3f3; border-radius: 50%; border-top: 16px solid #3498db; width: 80px; height: 80px; -webkit-animation: spin 2s linear infinite; animation: spin 2s linear infinite; } || requisitar("https://alura-scraping.herokuapp.com/produtos?_page="+1+"&_limit=10") ||You need to enable JavaScript to run this app.||Aguarde... ||Motors ||Hello World||Anúncios||Veículos de Luxo Novos e Usados - Todas as Marcas||246 veículos encontrados||Página 1 de 25||LAMBORGHINI AVENTADOR||USADO||Motor 1.8 16v||Ano 1993 - 55.286 km||► 4 X 4||► Câmera de estacionamento||► Controle de tração||► Sensor de estacionamento||...||Belo Horizonte - MG||R$ 338.000||BMW M2||USADO||Motor 3.0 32v||Ano 2018 - 83.447 km||► Câmera de estacio

# 3.5. Accessing the attributes of a tag

In [565]:
soup.img


<img alt="Alura" class="d-inline-block align-top" src="img/alura-logo.svg"/>

In [566]:
#um dict of all attributes of a tag {name_attr: value_of_attr, ...}
soup.img.attrs

{'alt': 'Alura',
 'class': ['d-inline-block', 'align-top'],
 'src': 'img/alura-logo.svg'}

When an attribute has more one value, the beatifulsoup return a list of values for this attributes, example the attribute class has two values, one list of values of this attribute.

In [567]:
#is possible work this attributes like a dict
soup.img.attrs.keys()

dict_keys(['src', 'class', 'alt'])

In [568]:
soup.img.attrs.values()

dict_values(['img/alura-logo.svg', ['d-inline-block', 'align-top'], 'Alura'])

In [569]:
#read one element of dict
soup.img['class']

['d-inline-block', 'align-top']

In [570]:
#get of value of the element of dict
soup.img.get('src')

'img/alura-logo.svg'

---
# <font color=green>4. SEARCHING WITH BEAUTIFULSOUP

# 4.1. The *find()* and *findAll()* methods

- ### *find(tag, attributes, recursive, text, **kwargs)*

- ### *findAll(tag, attributes, recursive, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all

> **Note:**
> - *findAll()* can also be used as *find_all()*

### *find()* method

Return the first tag founded

In [571]:
soup.find('img')

<img alt="Alura" class="d-inline-block align-top" src="img/alura-logo.svg"/>

In [572]:
# is the same
soup.img


<img alt="Alura" class="d-inline-block align-top" src="img/alura-logo.svg"/>

### *findAll()* method
Returns a list of found tags


In [573]:
soup.findAll('img')

[<img alt="Alura" class="d-inline-block align-top" src="img/alura-logo.svg"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/bmw-m2/bmw-m2-2970882__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/alfa/alfa-1823056__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/puech/puech-4055386__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-murcielago/lamborghini-murcielago-2872974__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/

### Command equivalent to the *find()* method

In [574]:
soup.findAll('img', limit =1)[0]

<img alt="Alura" class="d-inline-block align-top" src="img/alura-logo.svg"/>

### Shortcut to *findAll()* method

In [575]:
soup('img')

[<img alt="Alura" class="d-inline-block align-top" src="img/alura-logo.svg"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/bmw-m2/bmw-m2-2970882__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/alfa/alfa-1823056__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/puech/puech-4055386__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-murcielago/lamborghini-murcielago-2872974__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/

### Passing tag lists

In [576]:
soup.findAll(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

[<h5 class="modal-title" id="loadingModal_label"><span class="glyphicon glyphicon-refresh"></span>Aguarde... </h5>,
 <h4><b id="loadingModal_content"></b></h4>,
 <h1 class="sub-header">Veículos de Luxo Novos e Usados - Todas as Marcas</h1>]

### Using the *attributes* argument

In [577]:
soup.findAll('p')

[<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>,
 <p class="txt-category badge badge-secondary inline">USADO</p>,
 <p class="txt-motor">Motor 1.8 16v</p>,
 <p class="txt-description">Ano 1993 - 55.286 km</p>,
 <p class="txt-location">Belo Horizonte - MG</p>,
 <p class="txt-value">R$ 338.000</p>,
 <p class="txt-name inline">BMW M2</p>,
 <p class="txt-category badge badge-secondary inline">USADO</p>,
 <p class="txt-motor">Motor 3.0 32v</p>,
 <p class="txt-description">Ano 2018 - 83.447 km</p>,
 <p class="txt-location">Belo Horizonte - MG</p>,
 <p class="txt-value">R$ 346.000</p>,
 <p class="txt-name inline">ALFA</p>,
 <p class="txt-category badge badge-secondary inline">USADO</p>,
 <p class="txt-motor">Motor 1.8 16v</p>,
 <p class="txt-description">Ano 2004 - 19.722 km</p>,
 <p class="txt-location">Rio de Janeiro - RJ</p>,
 <p class="txt-value">R$ 480.000</p>,
 <p class="txt-name inline">PUECH</p>,
 <p class="txt-category badge badge-secondary inline">USADO</p>,
 <p class="txt-moto

### Searching for the content of a TAG

In [578]:
soup.findAll('p', {"class": "txt-value"})

[<p class="txt-value">R$ 338.000</p>,
 <p class="txt-value">R$ 346.000</p>,
 <p class="txt-value">R$ 480.000</p>,
 <p class="txt-value">R$ 133.000</p>,
 <p class="txt-value">R$ 175.000</p>,
 <p class="txt-value">R$ 239.000</p>,
 <p class="txt-value">R$ 115.000</p>,
 <p class="txt-value">R$ 114.000</p>,
 <p class="txt-value">R$ 75.000</p>,
 <p class="txt-value">R$ 117.000</p>]

In [579]:
soup.findAll('p', text = "Belo Horizonte - MG")

[<p class="txt-location">Belo Horizonte - MG</p>,
 <p class="txt-location">Belo Horizonte - MG</p>,
 <p class="txt-location">Belo Horizonte - MG</p>,
 <p class="txt-location">Belo Horizonte - MG</p>,
 <p class="txt-location">Belo Horizonte - MG</p>]

### Using attributes directly

In [580]:
soup.findAll('img', alt='Foto')

[<img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/bmw-m2/bmw-m2-2970882__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/alfa/alfa-1823056__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/puech/puech-4055386__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-murcielago/lamborghini-murcielago-2872974__340.jpg" width="220"/>,
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/aston-martin/aston-martin-2977916__340.jpg" width="220"/>,
 <img al

In [581]:
for item in soup.findAll('img', alt='Foto'):
  print(item['src'])

https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/bmw-m2/bmw-m2-2970882__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/alfa/alfa-1823056__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/puech/puech-4055386__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-murcielago/lamborghini-murcielago-2872974__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/aston-martin/aston-martin-2977916__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/tvr/tvr-2943925__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/excalibur/excalibur-2916730__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/mclaren/mclaren-2855240__340.jpg
htt

### Beware of the "class" attribute

In [582]:
soup.findAll('p', class_= "txt-value")

[<p class="txt-value">R$ 338.000</p>,
 <p class="txt-value">R$ 346.000</p>,
 <p class="txt-value">R$ 480.000</p>,
 <p class="txt-value">R$ 133.000</p>,
 <p class="txt-value">R$ 175.000</p>,
 <p class="txt-value">R$ 239.000</p>,
 <p class="txt-value">R$ 115.000</p>,
 <p class="txt-value">R$ 114.000</p>,
 <p class="txt-value">R$ 75.000</p>,
 <p class="txt-value">R$ 117.000</p>]

### Getting all the text content of a page

In [583]:
#return one list of all text in the page
soup.findAll(text=True)[:50]

['html',
 'Alura Motors',
 " /*Regra para a animacao*/ @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } } /*Mudando o tamanho do icone de resposta*/ div.glyphicon { color:#6B8E23; font-size: 38px; } /*Classe que mostra a animacao 'spin'*/ .loader { border: 16px solid #f3f3f3; border-radius: 50%; border-top: 16px solid #3498db; width: 80px; height: 80px; -webkit-animation: spin 2s linear infinite; animation: spin 2s linear infinite; } ",
 ' requisitar("https://alura-scraping.herokuapp.com/produtos?_page="+1+"&_limit=10") ',
 'You need to enable JavaScript to run this app.',
 'Aguarde... ',
 'Motors ',
 'Hello World',
 'Anúncios',
 'Veículos de Luxo Novos e Usados - Todas as Marcas',
 '246 veículos encontrados',
 'Página 1 de 25',
 'LAMBORGHINI AVENTADOR',
 'USADO',
 'Motor 1.8 16v',
 'Ano 1993 - 55.286 km',
 '► 4 X 4',
 '► Câmera de estacionamento',
 '► Controle de tração',
 '► Sensor de estacionamento',
 '...',
 'Belo Horizonte - MG',
 'R$ 338.000',

# 4.2. Other research methods

- ### *findParent(tag, attributes, text, **kwargs)*

- ### *findParents(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-parents-and-find-parent

> **Notes:**
> - *findParent()* e *findParents()* também podem ser utilizados como *find_parent()* e *find_parents()*, respectivamente.
---
- ### *findNextSibling(tag, attributes, text, **kwargs)*

- ### *findNextSiblings(tag, attributes, text, limit, **kwargs)*

- ### *findPreviousSibling(tag, attributes, text, **kwargs)*

- ### *findPreviousSiblings(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-next-siblings-and-find-next-sibling
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-previous-siblings-and-find-previous-sibling

> **Notes:**
> - *findNextSibling()*, *findNextSiblings()*, *findPreviousSibling()* and *findPreviousSiblings()* can also be used as *find_next_sibling()*, *find_next_siblings()*, *find_previous_sibling()* and *find_previous_siblings()*.
---
- ### *findNext(tag, attributes, text, **kwargs)*

- ### *findAllNext(tag, attributes, text, limit, **kwargs)*

- ### *findPrevious(tag, attributes, text, **kwargs)*

- ### *findAllPrevious(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all-next-and-find-next
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all-previous-and-find-previous

> **Notes:**
> - *findNext()*, *findAllNext()*, *findPrevious* e *findAllPrevious* can also be used as *find_next()*, *find_all_next()*, *find_previous()* and *find_all_previous()*.

## Sample HTML to illustrate the use of BeautifulSoup search methods

<img src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/BeautifulSoup-method.png" width=80%>

---
## Result

<html>
    <body>
        <div id=“container-a”>
            <h1>Título A</h1>
            <h2 class="ref-a">Sub título A</h2>
            <p>Texto de conteúdo A</p>
        </div>
        <div id=“container-b”>
            <h1>Título B</h1>
            <h2 class="ref-b">Sub título B</h2>
            <p>Texto de conteúdo B</p>
        </div>
    </body>
</html>

In [584]:
test_html = """
<html>
    <body>
        <div id=“container-a”>
            <h1>Título A</h1>
            <h2 class="ref-a">Sub título A</h2>
            <p>Texto de conteúdo A</p>
        </div>
        <div id=“container-b”>
            <h1>Título B</h1>
            <h2 class="ref-b">Sub título B</h2>
            <p>Texto de conteúdo B</p>
        </div>
    </body>
</html>
"""

### HTML string treatments

### Creating the BeautifulSoup object

In [585]:
soup = BeautifulSoup(test_html, 'html.parser')
soup


<html>
<body>
<div id="“container-a”">
<h1>Título A</h1>
<h2 class="ref-a">Sub título A</h2>
<p>Texto de conteúdo A</p>
</div>
<div id="“container-b”">
<h1>Título B</h1>
<h2 class="ref-b">Sub título B</h2>
<p>Texto de conteúdo B</p>
</div>
</body>
</html>

### Parents

In [586]:
soup.find('h2')

<h2 class="ref-a">Sub título A</h2>

In [587]:
soup.find('h2').find_parent('div')

<div id="“container-a”">
<h1>Título A</h1>
<h2 class="ref-a">Sub título A</h2>
<p>Texto de conteúdo A</p>
</div>

In [588]:
soup.find('h2').find_parents()

[<div id="“container-a”">
 <h1>Título A</h1>
 <h2 class="ref-a">Sub título A</h2>
 <p>Texto de conteúdo A</p>
 </div>, <body>
 <div id="“container-a”">
 <h1>Título A</h1>
 <h2 class="ref-a">Sub título A</h2>
 <p>Texto de conteúdo A</p>
 </div>
 <div id="“container-b”">
 <h1>Título B</h1>
 <h2 class="ref-b">Sub título B</h2>
 <p>Texto de conteúdo B</p>
 </div>
 </body>, <html>
 <body>
 <div id="“container-a”">
 <h1>Título A</h1>
 <h2 class="ref-a">Sub título A</h2>
 <p>Texto de conteúdo A</p>
 </div>
 <div id="“container-b”">
 <h1>Título B</h1>
 <h2 class="ref-b">Sub título B</h2>
 <p>Texto de conteúdo B</p>
 </div>
 </body>
 </html>, 
 <html>
 <body>
 <div id="“container-a”">
 <h1>Título A</h1>
 <h2 class="ref-a">Sub título A</h2>
 <p>Texto de conteúdo A</p>
 </div>
 <div id="“container-b”">
 <h1>Título B</h1>
 <h2 class="ref-b">Sub título B</h2>
 <p>Texto de conteúdo B</p>
 </div>
 </body>
 </html>]

In [589]:
soup.findAll('h2')

[<h2 class="ref-a">Sub título A</h2>, <h2 class="ref-b">Sub título B</h2>]

In [590]:
for item in soup.findAll('h2'):
  print(item.find_parent('div'))

<div id="“container-a”">
<h1>Título A</h1>
<h2 class="ref-a">Sub título A</h2>
<p>Texto de conteúdo A</p>
</div>
<div id="“container-b”">
<h1>Título B</h1>
<h2 class="ref-b">Sub título B</h2>
<p>Texto de conteúdo B</p>
</div>


## Siblings

In [591]:
soup.find('h2').findNextSibling()

<p>Texto de conteúdo A</p>

In [592]:
soup.find('h2').findPreviousSibling()

<h1>Título A</h1>

In [593]:
soup.find('p').findPreviousSiblings()

[<h2 class="ref-a">Sub título A</h2>, <h1>Título A</h1>]

## Next and Previous

In [594]:
soup.find('h2').findNext()

<p>Texto de conteúdo A</p>

In [595]:
soup.find('h2').findPrevious()

<h1>Título A</h1>

In [596]:
soup.find('h2').findAllNext()

[<p>Texto de conteúdo A</p>, <div id="“container-b”">
 <h1>Título B</h1>
 <h2 class="ref-b">Sub título B</h2>
 <p>Texto de conteúdo B</p>
 </div>, <h1>Título B</h1>, <h2 class="ref-b">Sub título B</h2>, <p>Texto de conteúdo B</p>]

# <font color=green>5. CAR SITE WEB SCRAPING - GETTING THE DATA OF AN ADVERTISING

# 5.1. Identifying and Selecting Data in HTML

### Getting the HTML and creating the BeautifulSoup object

In [597]:
response = urlopen('https://alura-site-scraping.herokuapp.com/index.php')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')


### Creating variable to store information

In [598]:
cards = [] # list of cars
card = {}  #dic each car

### Getting the data of the first CARD

In [599]:
announcement = soup.find('div', {'class': 'well card'})
announcement

<div class="well card">
<div class="col-md-3 image-card">
<img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>
</div>
<div class="col-md-6 body-card">
<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>
<p class="txt-category badge badge-secondary inline">USADO</p>
<p class="txt-motor">Motor 1.8 16v</p>
<p class="txt-description">Ano 1993 - 55.286 km</p>
<ul class="lst-items">
<li class="txt-items">► 4 X 4</li>
<li class="txt-items">► Câmera de estacionamento</li>
<li class="txt-items">► Controle de tração</li>
<li class="txt-items">► Sensor de estacionamento</li>
<li class="txt-items">...</li>
</ul>
<p class="txt-location">Belo Horizonte - MG</p>
</div>
<div class="col-md-3 value-card">
<div class="value">
<p class="txt-value">R$ 338.000</p>
</div>
</div>
</div>

# 5.2. Getting the VALUE of the advertised vehicle

In [600]:
announcement.find('div', {'class': 'value-card'})

<div class="col-md-3 value-card">
<div class="value">
<p class="txt-value">R$ 338.000</p>
</div>
</div>

In [601]:
announcement.find('p', {'class': 'txt-value'}).getText()

'R$ 338.000'

### <font color=red>Summary

In [602]:
# Value of car
card['value'] = announcement.find('p', {'class': 'txt-value'}).getText()


# 5.3. Obtaining information about the advertised vehicle

In [603]:
announcement.find('div', {'class': 'body-card'}).findAll('p')

[<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>,
 <p class="txt-category badge badge-secondary inline">USADO</p>,
 <p class="txt-motor">Motor 1.8 16v</p>,
 <p class="txt-description">Ano 1993 - 55.286 km</p>,
 <p class="txt-location">Belo Horizonte - MG</p>]

In [604]:
infos = announcement.find('div', {'class': 'body-card'}).findAll('p')

In [605]:
for info in infos:
    print(info)

<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>
<p class="txt-category badge badge-secondary inline">USADO</p>
<p class="txt-motor">Motor 1.8 16v</p>
<p class="txt-description">Ano 1993 - 55.286 km</p>
<p class="txt-location">Belo Horizonte - MG</p>


In [606]:
for info in infos:
    print(info.get('class'))

['txt-name', 'inline']
['txt-category', 'badge', 'badge-secondary', 'inline']
['txt-motor']
['txt-description']
['txt-location']


In [607]:
for info in infos:
    print(info.get('class'), ' - ', info.get_text())

['txt-name', 'inline']  -  LAMBORGHINI AVENTADOR
['txt-category', 'badge', 'badge-secondary', 'inline']  -  USADO
['txt-motor']  -  Motor 1.8 16v
['txt-description']  -  Ano 1993 - 55.286 km
['txt-location']  -  Belo Horizonte - MG


In [608]:
for info in infos:
    print(info.get('class')[0], ' - ', info.get_text())

txt-name  -  LAMBORGHINI AVENTADOR
txt-category  -  USADO
txt-motor  -  Motor 1.8 16v
txt-description  -  Ano 1993 - 55.286 km
txt-location  -  Belo Horizonte - MG


In [609]:
for info in infos:
    print(info.get('class')[0].split('-'), ' - ', info.get_text())

['txt', 'name']  -  LAMBORGHINI AVENTADOR
['txt', 'category']  -  USADO
['txt', 'motor']  -  Motor 1.8 16v
['txt', 'description']  -  Ano 1993 - 55.286 km
['txt', 'location']  -  Belo Horizonte - MG


In [610]:
for info in infos:
    card[info.get('class')[0].split('-')[-1]] = info.get_text()
card

{'category': 'USADO',
 'description': 'Ano 1993 - 55.286 km',
 'location': 'Belo Horizonte - MG',
 'motor': 'Motor 1.8 16v',
 'name': 'LAMBORGHINI AVENTADOR',
 'value': 'R$ 338.000'}

### <font color=red>Summary

In [611]:
# Informações
infos = announcement.find('div', {'class': 'body-card'}).findAll('p')
for info in infos:
    card[info.get('class')[0].split('-')[-1]] = info.get_text()

# 5.4. Getting the ACCESSORIES of the advertised vehicle

In [612]:
announcement.find('div', {'class': 'body-card'}).ul.findAll('li')

[<li class="txt-items">► 4 X 4</li>,
 <li class="txt-items">► Câmera de estacionamento</li>,
 <li class="txt-items">► Controle de tração</li>,
 <li class="txt-items">► Sensor de estacionamento</li>,
 <li class="txt-items">...</li>]

In [613]:
items = announcement.find('div', {'class': 'body-card'}).ul.findAll('li')

In [614]:
items.pop() #remove the last item

<li class="txt-items">...</li>

In [615]:
for item in items:
  print(item.get_text().replace('► ', ''))

4 X 4
Câmera de estacionamento
Controle de tração
Sensor de estacionamento


In [616]:
accessories = []


In [617]:
for item in items:
  accessories.append(item.get_text().replace('► ', ''))

In [618]:
card['itens'] = accessories

In [619]:
card

{'category': 'USADO',
 'description': 'Ano 1993 - 55.286 km',
 'itens': ['4 X 4',
  'Câmera de estacionamento',
  'Controle de tração',
  'Sensor de estacionamento'],
 'location': 'Belo Horizonte - MG',
 'motor': 'Motor 1.8 16v',
 'name': 'LAMBORGHINI AVENTADOR',
 'value': 'R$ 338.000'}

### <font color=red>Summary

In [620]:
# Acessórios
items = announcement.find('div', {'class': 'body-card'}).ul.findAll('li')
items.pop() #remove the last item
accessories = []
for item in items:
  accessories.append(item.get_text().replace('► ', ''))

card['items'] = accessories
card



{'category': 'USADO',
 'description': 'Ano 1993 - 55.286 km',
 'items': ['4 X 4',
  'Câmera de estacionamento',
  'Controle de tração',
  'Sensor de estacionamento'],
 'itens': ['4 X 4',
  'Câmera de estacionamento',
  'Controle de tração',
  'Sensor de estacionamento'],
 'location': 'Belo Horizonte - MG',
 'motor': 'Motor 1.8 16v',
 'name': 'LAMBORGHINI AVENTADOR',
 'value': 'R$ 338.000'}

# 5.5 Creating a DataFrame with the data collected 

In [621]:
import pandas as pd

In [622]:
dataset = pd.DataFrame.from_dict(card, orient='index')
dataset

,0
value,R$ 338.000
name,LAMBORGHINI AVENTADOR
category,USADO
motor,Motor 1.8 16v
description,Ano 1993 - 55.286 km
location,Belo Horizonte - MG
itens,"[4 X 4, Câmera de estacionamento, Controle de ..."
items,"[4 X 4, Câmera de estacionamento, Controle de ..."


The dataset is wrong, its necessary the dataset similar a row in a table, this happened because the content of key "items". Is possible solve this problem with .T "transpose" like below.

In [623]:
dataset = pd.DataFrame.from_dict(card, orient='index').T
dataset

,value,name,category,motor,description,location,itens,items
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"[4 X 4, Câmera de estacionamento, Controle de ...","[4 X 4, Câmera de estacionamento, Controle de ..."


In [624]:
dataset.to_csv('./sample_data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')

# 5.6. Getting the ad PHOTO

In [625]:
announcement

<div class="well card">
<div class="col-md-3 image-card">
<img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>
</div>
<div class="col-md-6 body-card">
<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>
<p class="txt-category badge badge-secondary inline">USADO</p>
<p class="txt-motor">Motor 1.8 16v</p>
<p class="txt-description">Ano 1993 - 55.286 km</p>
<ul class="lst-items">
<li class="txt-items">► 4 X 4</li>
<li class="txt-items">► Câmera de estacionamento</li>
<li class="txt-items">► Controle de tração</li>
<li class="txt-items">► Sensor de estacionamento</li>
<li class="txt-items">...</li>
</ul>
<p class="txt-location">Belo Horizonte - MG</p>
</div>
<div class="col-md-3 value-card">
<div class="value">
<p class="txt-value">R$ 338.000</p>
</div>
</div>
</div>

In [626]:
image = announcement.find('div', {'class': 'image-card'}).img

In [627]:
image.get('src')

'https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg'

### Viewing the PHOTO on the notebook

In [628]:
from IPython.core.display import display, HTML
display(HTML(str(announcement.find('div', {'class': 'image-card'}).img)))

### Routine to access and save the ad PHOTO

## https://docs.python.org/3/library/urllib.request.html#urllib.request.urlretrieve

In [629]:
from urllib.request import urlretrieve

In [630]:
image.get('src').split('/')[-1]

'lamborghini-aventador-2932196__340.jpg'

### <font color=red>Summary

In [631]:
from urllib.request import urlretrieve

urlretrieve(image.get('src'), './sample_data/' + image.get('src').split('/')[-1])

('./sample_data/lamborghini-aventador-2932196__340.jpg',
 <http.client.HTTPMessage at 0x7f7b6c1fa690>)

# <font color=green>6. WEBSITE WEB SCRAPING - GETTING THE DATA OF ALL ADS FROM A PAGE

# 6.1. Identifying information in HTML

In [632]:
# number of announcements
len(soup.find('div', {'id' : 'container-cards'}).findAll('div', class_='card'))

10

In [633]:
announcements = soup.find('div', {'id' : 'container-cards'}).findAll('div', class_='card')

In [634]:
for item in announcements:
  print(str(item) + '\n\n')

<div class="well card">
<div class="col-md-3 image-card">
<img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>
</div>
<div class="col-md-6 body-card">
<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>
<p class="txt-category badge badge-secondary inline">USADO</p>
<p class="txt-motor">Motor 1.8 16v</p>
<p class="txt-description">Ano 1993 - 55.286 km</p>
<ul class="lst-items">
<li class="txt-items">► 4 X 4</li>
<li class="txt-items">► Câmera de estacionamento</li>
<li class="txt-items">► Controle de tração</li>
<li class="txt-items">► Sensor de estacionamento</li>
<li class="txt-items">...</li>
</ul>
<p class="txt-location">Belo Horizonte - MG</p>
</div>
<div class="col-md-3 value-card">
<div class="value">
<p class="txt-value">R$ 338.000</p>
</div>
</div>
</div>


<div class="well card">
<div class="col-md-3 image-card">
<img alt="Foto" height="155" src="ht

# 6.2. Creating a scraping routine

In [635]:
# Importando bibliotecas
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

# Declarando variável cards
cards = []

# Obtendo o HTML
response = urlopen('https://alura-site-scraping.herokuapp.com/index.php')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

# Obtendo as TAGs de interesse
announcement = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card")

# Coletando as informações dos CARDS
for announcement in announcements:
    card = {}

    # Valor
    card['value'] = announcement.find('p', {'class': 'txt-value'}).getText()

    # Informações
    infos = announcement.find('div', {'class': 'body-card'}).findAll('p')
    for info in infos:
        card[info.get('class')[0].split('-')[-1]] = info.get_text()

    # Acessórios
    items = announcement.find('div', {'class': 'body-card'}).ul.findAll('li')
    items.pop()
    acessorios = []
    for item in items:
        acessorios.append(item.get_text().replace('► ', ''))
    card['items'] = acessorios

     # Adicionando resultado a lista cards
    cards.append(card)

    # Imagens
    image = announcement.find('div', {'class': 'image-card'}).img
    urlretrieve(image.get('src'), './sample_data/' + image.get('src').split('/')[-1])

# Criando um DataFrame com os resultados
dataset = pd.DataFrame(cards)
dataset.to_csv('./sample_data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset

,value,name,category,motor,description,location,items,opportunity
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"[4 X 4, Câmera de estacionamento, Controle de ...",NaN
1,R$ 346.000,BMW M2,USADO,Motor 3.0 32v,Ano 2018 - 83.447 km,Belo Horizonte - MG,"[Câmera de estacionamento, Controle de estabil...",NaN
2,R$ 480.000,ALFA,USADO,Motor 1.8 16v,Ano 2004 - 19.722 km,Rio de Janeiro - RJ,"[Central multimídia, Bancos de couro, Rodas de...",NaN
3,R$ 133.000,PUECH,USADO,Motor Diesel V8,Ano 1992 - 34.335 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Rodas de liga, C...",NaN
4,R$ 175.000,LAMBORGHINI MURCIELAGO,USADO,Motor 1.0 8v,Ano 1991 - 464 km,Belo Horizonte - MG,"[Central multimídia, Teto panorâmico, Sensor c...",NaN
5,R$ 239.000,ASTON MARTIN,USADO,Motor Diesel V6,Ano 2004 - 50.189 km,Belo Horizonte - MG,"[Painel digital, Controle de tração, Teto pano...",OPORTUNIDADE
6,R$ 115.000,TVR,USADO,Motor 4.0 Turbo,Ano 2014 - 17.778 km,Belo Horizonte - MG,"[4 X 4, Teto panorâmico, Central multimídia, C...",NaN
7,R$ 114.000,EXCALIBUR,USADO,Motor 3.0 32v,Ano 2009 - 81.251 km,Rio de Janeiro - RJ,"[Painel digital, Câmbio automático, Sensor de ...",NaN
8,R$ 75.000,MCLAREN,NOVO,Motor Diesel,Ano 2019 - 0 km,São Paulo - SP,"[Central multimídia, Câmera de estacionamento,...",NaN
9,R$ 117.000,TOYOTA,USADO,Motor 4.0 Turbo,Ano 1999 - 12.536 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Piloto automátic...",OPORTUNIDADE


In [636]:
for image in soup('img'):
  print(image.get('src'))

img/alura-logo.svg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/bmw-m2/bmw-m2-2970882__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/alfa/alfa-1823056__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/puech/puech-4055386__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-murcielago/lamborghini-murcielago-2872974__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/aston-martin/aston-martin-2977916__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/tvr/tvr-2943925__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/excalibur/excalibur-2916730__340.jpg
https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/mclaren/mclaren-2

# <font color=green>7. WEB SCRAPING THE SITE - GETTING THE DATA OF ALL ADVERTISING ON THE SITE

# 7.1. Identifying information about number of pages in HTML

In [637]:
soup.find('span', class_="info-pages")

<span class="info-pages">Página 1 de 25</span>

In [638]:
soup.find('span', class_="info-pages").get_text()

'Página 1 de 25'

In [639]:
soup.find('span', class_="info-pages").get_text().split()

['Página', '1', 'de', '25']

In [640]:
soup.find('span', class_="info-pages").get_text().split()[-1]

'25'

In [641]:
int(soup.find('span', class_="info-pages").get_text().split()[-1])

25

# 7.2. Creating a scraping routine

In [642]:
# Importando bibliotecas
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

# Declarando variável cards
cards = []

# Obtendo o HTML
response = urlopen('https://alura-site-scraping.herokuapp.com/index.php')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
number_of_pages = int(soup.find('span', class_="info-pages").get_text().split()[-1])

for i in range(1,number_of_pages+1):
  response = urlopen('https://alura-site-scraping.herokuapp.com/index.php?page={}'.format(i))
  html = response.read().decode('utf-8')
  soup = BeautifulSoup(html, 'html.parser')
    

  # Obtendo as TAGs de interesse
  announcements = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card")

  # Coletando as informações dos CARDS
  for announcement in announcements:
      card = {}

      # Valor
      card['value'] = announcement.find('p', {'class': 'txt-value'}).getText()

      # Informações
      infos = announcement.find('div', {'class': 'body-card'}).findAll('p')
      for info in infos:
          card[info.get('class')[0].split('-')[-1]] = info.get_text()

      # Acessórios
      items = announcement.find('div', {'class': 'body-card'}).ul.findAll('li')
      items.pop()
      acessorios = []
      for item in items:
          acessorios.append(item.get_text().replace('► ', ''))
      card['items'] = acessorios

      # Adicionando resultado a lista cards
      cards.append(card)

      # Imagens
      image = announcement.find('div', {'class': 'image-card'}).img
      urlretrieve(image.get('src'), './sample_data/' + image.get('src').split('/')[-1])

# Criando um DataFrame com os resultados
dataset = pd.DataFrame(cards)
dataset.to_csv('./sample_data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset

,value,name,category,motor,description,location,items,opportunity
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"[4 X 4, Câmera de estacionamento, Controle de ...",NaN
1,R$ 346.000,BMW M2,USADO,Motor 3.0 32v,Ano 2018 - 83.447 km,Belo Horizonte - MG,"[Câmera de estacionamento, Controle de estabil...",NaN
2,R$ 480.000,ALFA,USADO,Motor 1.8 16v,Ano 2004 - 19.722 km,Rio de Janeiro - RJ,"[Central multimídia, Bancos de couro, Rodas de...",NaN
3,R$ 133.000,PUECH,USADO,Motor Diesel V8,Ano 1992 - 34.335 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Rodas de liga, C...",NaN
4,R$ 175.000,LAMBORGHINI MURCIELAGO,USADO,Motor 1.0 8v,Ano 1991 - 464 km,Belo Horizonte - MG,"[Central multimídia, Teto panorâmico, Sensor c...",NaN
...,...,...,...,...,...,...,...,...
241,R$ 489.000,SUV REAR TIRE,USADO,Motor 3.0 32v,Ano 1998 - 74.292 km,São Paulo - SP,"[Câmera de estacionamento, Rodas de liga, Sens...",NaN
242,R$ 427.000,ANTIQUE,NOVO,Motor 2.0 16v,Ano 2019 - 0 km,Belo Horizonte - MG,"[Bancos de couro, Freios ABS, Sensor de estaci...",NaN
243,R$ 203.000,SPORT,USADO,Motor 2.0 16v,Ano 2001 - 102.776 km,Belo Horizonte - MG,"[Sensor crepuscular, Sensor de chuva, Vidros e...",NaN
244,R$ 474.000,IMPERIAL,USADO,Motor 1.8 16v,Ano 2011 - 101.787 km,Belo Horizonte - MG,"[Painel digital, Travas elétricas, Sensor de c...",OPORTUNIDADE
